<a href="https://colab.research.google.com/github/yulinkai/Colab/blob/main/Transforfmer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U datasets
from datasets import load_dataset

raw_datasets = load_dataset("Helsinki-NLP/opus-100", "en-zh")
split_datasets = raw_datasets["train"].train_test_split(train_size=9000, test_size=1000, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/355k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/143M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [2]:
split_datasets["train"][200]["translation"]

{'en': 'For example “Land for those who work it, not just for those who can buy it”, April 2002, http://www.foodfirst.org/progs/global/trade/worldbankseminar.html.',
 'zh': '例如“耕者有其田，而不只是属于买得起土地者”，2002年4月，http://www.foodfirst.org/ progs/global/trade/worldbankseminar.html。'}

In [3]:
from transformers import pipeline, AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-zh"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="tf")
translator = pipeline("translation", model=model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/806k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/805k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Device set to use cuda:0


model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

In [4]:
translator("For example “Land for those who work it, not just for those who can buy it”, April 2002, http://www.foodfirst.org/progs/global/trade/worldbankseminar.html.")

[{'translation_text': '例如,2002年4月,http://www.foodfirst.org/progs/global/trade/worldbankseminar.html。'}]

In [9]:
max_input_length = 128
max_target_length = 128


def preprocess_function(examples):
    """預處理輸入數據用於機器翻譯任務

    此函數將原始的英文-中文翻譯對進行預處理,包括分詞和截斷等操作,
    使其符合模型輸入要求的格式。

    Args:
        examples (dict): 包含翻譯對的字典
            - translation: 包含英文(en)和中文(zh)翻譯對的列表

    Returns:
        dict: 處理後的模型輸入字典
            - input_ids: 英文句子的token id序列
            - attention_mask: 注意力遮罩
            - labels: 中文句子的token id序列
    """
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["zh"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

In [10]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])

In [13]:
!pip install sacrebleu
!pip install evaluate

import evaluate

metric = evaluate.load("sacrebleu")

import numpy as np


def compute_metrics(eval_preds):
    """計算模型預測結果的BLEU評分指標

    此函數用於計算機器翻譯模型的BLEU(Bilingual Evaluation Understudy)分數,
    通過比較模型生成的翻譯結果與參考翻譯的相似度來評估翻譯品質。

    Args:
        eval_preds (tuple): 包含預測結果和標籤的元組
            - preds: 模型的預測輸出,shape為(batch_size, seq_length)
            - labels: 真實標籤,shape為(batch_size, seq_length)

    Returns:
        dict: 包含BLEU分數的字典
            - bleu: BLEU評分結果(0-100的浮點數)
    """
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [14]:
!pip install -U transformers

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

args = Seq2SeqTrainingArguments(
    output_dir="finetuned-model",                 # 模型儲存路徑
    save_strategy="epoch",                        # 每個 epoch 儲存一次模型
    learning_rate=2e-5,                           # 學習率設定
    per_device_train_batch_size=32,               # 訓練時每個設備的批次大小
    per_device_eval_batch_size=64,                # 評估時每個設備的批次大小
    weight_decay=0.01,                            # 權重衰減率，用於正則化
    save_total_limit=3,                           # 最多保存最近3個檢查點
    num_train_epochs=3,                           # 訓練總輪數
    predict_with_generate=True,                   # 使用生成式預測進行評估
    fp16=True,                                    # 啟用16位元浮點數訓練以加速
    push_to_hub=False,                            # 不將模型推送到 Hugging Face Hub
    report_to=[]                                  # 不使用任何報告工具(如 wandb)
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-14-367fb247d94d>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"
trainer.evaluate(max_length=max_target_length)

{'eval_loss': 1.8458378314971924,
 'eval_model_preparation_time': 0.0053,
 'eval_bleu': 10.95379118079008,
 'eval_runtime': 79.827,
 'eval_samples_per_second': 12.527,
 'eval_steps_per_second': 0.2}

In [16]:
trainer.train()

Step,Training Loss
500,1.779300


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=846, training_loss=1.6967753236739067, metrics={'train_runtime': 165.1601, 'train_samples_per_second': 163.478, 'train_steps_per_second': 5.122, 'total_flos': 576602712834048.0, 'train_loss': 1.6967753236739067, 'epoch': 3.0})

In [17]:
trainer.evaluate(max_length=max_target_length)

{'eval_loss': 1.7207732200622559,
 'eval_model_preparation_time': 0.0053,
 'eval_bleu': 11.91901022763684,
 'eval_runtime': 67.1952,
 'eval_samples_per_second': 14.882,
 'eval_steps_per_second': 0.238,
 'epoch': 3.0}

In [18]:
from transformers import pipeline

translation_pipeline = pipeline("translation", model=trainer.model, tokenizer=tokenizer)

# 測試翻譯英文句子
sentence = "For example “Land for those who work it, not just for those who can buy it”, April 2002, http://www.foodfirst.org/progs/global/trade/worldbankseminar.html."
translated = translation_pipeline(sentence)
print(translated[0]['translation_text'])

Device set to use cuda:0


例如,“工作者的土地,而不仅仅是买得起土地的人”,2002年4月,http://www.foodfirst.org/progs/global/trade/worldbankseminar.html。
